**Goal : Identify bike stations that have potential for expansion by comparing the growth of bike usage at different stations and by determining the popular routes**

**Methodology:** 
- First, we performed some exploratory analysis by looking at which stations were the most popular starting and ending points within the trips data. Next, we looked at the trips that occurred at each bike station over a day and compared it to the bike station's capacity. If the capacity was exceeded atleast 40 times in a year, we marked that station as a good candidate for expansion. 
- Finally, we used a 1km buffer zone around the bike station, and marked those bike stations which are close to TTC stations and schools as being candidates for expansion.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import geopandas as gpd

!pip install folium
import folium
from folium import Circle
from folium.plugins import HeatMap
from folium import GeoJson
from folium import Marker

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import 'trips_data'
trips_data = pd.read_csv('trips_data (1).csv')
trips_data.head(10)

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,User Type
0,712441,274,7006.0,2017-01-01 00:03:00-05:00,Bay St / College St (East Side),7021.0,2017-01-01 00:08:00-05:00,Bay St / Albert St,Member
1,712442,538,7046.0,2017-01-01 00:03:00-05:00,Niagara St / Richmond St W,7147.0,2017-01-01 00:12:00-05:00,King St W / Fraser Ave,Member
2,712443,992,7048.0,2017-01-01 00:05:00-05:00,Front St W / Yonge St (Hockey Hall of Fame),7089.0,2017-01-01 00:22:00-05:00,Church St / Wood St,Member
3,712444,1005,7177.0,2017-01-01 00:09:00-05:00,East Liberty St / Pirandello St,7202.0,2017-01-01 00:26:00-05:00,Queen St W / York St (City Hall),Member
4,712445,645,7203.0,2017-01-01 00:14:00-05:00,Bathurst St / Queens Quay W,7010.0,2017-01-01 00:25:00-05:00,King St W / Spadina Ave,Member
5,712446,660,7193.0,2017-01-01 00:15:00-05:00,Queen St W / Gladstone Ave,7123.0,2017-01-01 00:26:00-05:00,424 Wellington St W,Member
6,712447,625,7003.0,2017-01-01 00:15:00-05:00,Madison Ave / Bloor St W,7003.0,2017-01-01 00:25:00-05:00,Madison Ave / Bloor St W,Member
7,712448,348,7000.0,2017-01-01 00:17:00-05:00,Fort York Blvd / Capreol Ct,7070.0,2017-01-01 00:23:00-05:00,25 York St – Union Station South,Member
8,712449,288,7078.0,2017-01-01 00:24:00-05:00,College St / Major St,7061.0,2017-01-01 00:29:00-05:00,Bloor St / Brunswick Ave,Member
9,712450,614,7023.0,2017-01-01 00:30:00-05:00,College St / Borden St,7102.0,2017-01-01 00:40:00-05:00,Nelson St / Duncan St,Casual


In [3]:
# Import Station Data
station_data = pd.read_csv('stations.csv')
station_data.head(10)

,station_id,name,lat,lon
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954
1,7001,Wellesley Station Green P,43.664964,-79.383550
2,7002,St. George St / Bloor St W,43.667333,-79.399429
3,7003,Madison Ave / Bloor St W,43.667158,-79.402761
4,7004,University Ave / Elm St,43.656518,-79.389099
5,7005,King St W / York St,43.648001,-79.383177
6,7006,Bay St / College St (East Side),43.660439,-79.385525
7,7007,College St / Huron St,43.658148,-79.398167
8,7008,Wellesley St / Queen's Park Cres,43.663376,-79.392125
9,7009,King St E / Jarvis St,43.650325,-79.372287


In [4]:
# Most popular Starting Station
Start_Station = trips_data.groupby(by=['Start Station Name','Start Station Id'])['Start Station Id'].count().sort_values(ascending = False)


In [5]:
print('10 most popular Starting Stations are : ')
Popular_start_stations = Start_Station.to_frame()

Popular_start_stations.rename(columns={'Start Station Id' : 'Number of Trips'}).head(10)

10 most popular Starting Stations are : 


,,Number of Trips
Start Station Name,Start Station Id,
York St / Queens Quay W,7076.0,158085
Bay St / Queens Quay W (Ferry Terminal),7016.0,127324
Lake Shore Blvd W / Ontario Dr,7242.0,112313
HTO Park (Queens Quay W),7175.0,111271
Bathurst St/Queens Quay(Billy Bishop Airport),7203.0,106753
Ontario Place Blvd / Lake Shore Blvd W (East),7171.0,105872
Bay St / College St (East Side),7006.0,104817
Bay St / Wellesley St W,7030.0,97239
Queens Quay / Yonge St,7168.0,95414


In [6]:
# Most popular End Station
End_Station = trips_data.groupby(by=['End Station Name','End Station Id'])['End Station Id'].count().sort_values(ascending = False)


In [7]:
print('10 most popular End Stations are : ')

Popular_end_stations = End_Station.to_frame()

Popular_end_stations.rename(columns={'End Station Id' : 'Number of Trips'}).head(10)

10 most popular End Stations are : 


,,Number of Trips
End Station Name,End Station Id,
York St / Queens Quay W,7076.0,174399
Bay St / Queens Quay W (Ferry Terminal),7016.0,136263
HTO Park (Queens Quay W),7175.0,123705
Union Station,7033.0,118413
Bathurst St/Queens Quay(Billy Bishop Airport),7203.0,115499
Lake Shore Blvd W / Ontario Dr,7242.0,111601
Ontario Place Blvd / Lake Shore Blvd W (East),7171.0,106634
Bay St / College St (East Side),7006.0,104946
Queens Quay / Yonge St,7168.0,104195


In [8]:
#Most Popular Routes
temp = trips_data[trips_data['Start Station Id'] != trips_data['End Station Id']]
routes = temp.groupby(['Start Station Id','End Station Id']).size().sort_values(ascending=False)
routes = routes.to_frame()


In [9]:
Popular_routes = routes.reset_index()
Popular_routes = Popular_routes[:11]

Popular_routes = Popular_routes.join(station_data.set_index('station_id')['name'], on='Start Station Id')
Popular_routes = Popular_routes.rename(columns={"name": "Start Station Name"})
Popular_routes = Popular_routes.join(station_data.set_index('station_id')['name'], on='End Station Id')
Popular_routes = Popular_routes.rename(columns={"name": "End Station Name", 0 : "Number of Rides"})



In [10]:
Popular_routes

,Start Station Id,End Station Id,Number of Rides,Start Station Name,End Station Name
0,7344.0,7354.0,9590,Cherry Beach,Tommy Thompson Park (Leslie Street Spit)
1,7354.0,7344.0,8615,Tommy Thompson Park (Leslie Street Spit),Cherry Beach
2,7203.0,7076.0,7838,Bathurst St/Queens Quay(Billy Bishop Airport),York St / Queens Quay W
3,7076.0,7203.0,7069,York St / Queens Quay W,Bathurst St/Queens Quay(Billy Bishop Airport)
4,7059.0,7033.0,7056,Front St W / Blue Jays Way,Union Station
5,7171.0,7242.0,6959,Ontario Place Blvd / Lake Shore Blvd W (East),Lake Shore Blvd W / Ontario Dr
6,7289.0,7288.0,6096,Humber Bay Shores Park East,Humber Bay Shores Park West
7,7242.0,7222.0,5916,Lake Shore Blvd W / Ontario Dr,Sunnyside - Gus Ryder Pool
8,7430.0,7289.0,5812,Marilyn Bell Park Tennis Court,Humber Bay Shores Park East
9,7261.0,7344.0,5693,Queens Quay E / Lower Sherbourne St,Cherry Beach


In [11]:
# Heatmap of all the Station distributions
map1 = folium.Map(location=[43.639832, -79.395954], 
                   tiles='cartodbpositron', 
                   zoom_start=10)

df = station_data[['lat', 'lon']]
HeatMap(df, radius=10).add_to(map1)

# Display map
map1

In [12]:
#Trips_data dataframe to Stations_dataframe
df = trips_data.loc[trips_data['Start Station Id'].notnull()]
df = df.join(station_data.set_index('station_id'), how='left', on='Start Station Id')
df = df.rename(columns={"name": "Start Station name", "lon": "Start lon", 'lat': 'Start lat'})

In [13]:
df.head(10)

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,User Type,Start Station name,Start lat,Start lon
0,712441,274,7006.0,2017-01-01 00:03:00-05:00,Bay St / College St (East Side),7021.0,2017-01-01 00:08:00-05:00,Bay St / Albert St,Member,Bay St / College St (East Side),43.660439,-79.385525
1,712442,538,7046.0,2017-01-01 00:03:00-05:00,Niagara St / Richmond St W,7147.0,2017-01-01 00:12:00-05:00,King St W / Fraser Ave,Member,Niagara St / Richmond St W,43.645340,-79.409597
2,712443,992,7048.0,2017-01-01 00:05:00-05:00,Front St W / Yonge St (Hockey Hall of Fame),7089.0,2017-01-01 00:22:00-05:00,Church St / Wood St,Member,Front St W / Yonge St (Hockey Hall of Fame),43.646144,-79.377962
3,712444,1005,7177.0,2017-01-01 00:09:00-05:00,East Liberty St / Pirandello St,7202.0,2017-01-01 00:26:00-05:00,Queen St W / York St (City Hall),Member,East Liberty St / Pirandello St,43.638611,-79.414722
4,712445,645,7203.0,2017-01-01 00:14:00-05:00,Bathurst St / Queens Quay W,7010.0,2017-01-01 00:25:00-05:00,King St W / Spadina Ave,Member,Bathurst St/Queens Quay(Billy Bishop Airport),43.635492,-79.398253
5,712446,660,7193.0,2017-01-01 00:15:00-05:00,Queen St W / Gladstone Ave,7123.0,2017-01-01 00:26:00-05:00,424 Wellington St W,Member,Queen St W / Gladstone Ave,43.643056,-79.427500
6,712447,625,7003.0,2017-01-01 00:15:00-05:00,Madison Ave / Bloor St W,7003.0,2017-01-01 00:25:00-05:00,Madison Ave / Bloor St W,Member,Madison Ave / Bloor St W,43.667158,-79.402761
7,712448,348,7000.0,2017-01-01 00:17:00-05:00,Fort York Blvd / Capreol Ct,7070.0,2017-01-01 00:23:00-05:00,25 York St – Union Station South,Member,Fort York Blvd / Capreol Ct,43.639832,-79.395954
8,712449,288,7078.0,2017-01-01 00:24:00-05:00,College St / Major St,7061.0,2017-01-01 00:29:00-05:00,Bloor St / Brunswick Ave,Member,College St / Major St,43.657600,-79.403200
9,712450,614,7023.0,2017-01-01 00:30:00-05:00,College St / Borden St,7102.0,2017-01-01 00:40:00-05:00,Nelson St / Duncan St,Casual,College St / Borden St,43.657100,-79.405600


In [14]:
#Usage of each Station
df = df[['Start Station Name', 'Start Station Id','Start lat', 'Start lon']]
df = df.loc[df['Start lat'].notnull()]
df = df.loc[df['Start lon'].notnull()]

unique_stations = df['Start Station Id'].value_counts().to_frame(name='proportion')
unique_stations['Usage'] = unique_stations['proportion']*100/df.shape[0]

unique_stations = unique_stations.join(station_data.set_index('station_id'), how='left')

In [15]:
unique_stations[['name','lat','lon','Usage']]

,name,lat,lon,Usage
7076.0,York St / Queens Quay W,43.640132,-79.380464,1.041582
7016.0,Bay St / Queens Quay W (Ferry Terminal),43.640978,-79.376785,0.838905
7242.0,Lake Shore Blvd W / Ontario Dr,43.630254,-79.420317,0.740002
7175.0,HTO Park (Queens Quay W),43.637960,-79.387502,0.733136
7171.0,Ontario Place Blvd / Lake Shore Blvd W (East),43.632244,-79.410810,0.721092
...,...,...,...,...
7678.0,York University Station (South) - SMART,43.773450,-79.500200,0.005989
7631.0,Florence Gell Park,43.661547,-79.498398,0.005264
7679.0,York University Station (North),43.774100,-79.500500,0.005172
7624.0,Guildwood GO Station (South),43.754978,-79.197129,0.004289


In [16]:
# Plot the usage of stations
map2 = folium.Map(location=[43.639832, -79.395954], 
                   tiles='cartodbpositron', 
                   zoom_start=10)

st_prop = unique_stations[['lat','lon','proportion']].values.tolist()
HeatMap(st_prop,min_opacity=0.1, 
                max_opacity=0.5, 
                radius=25,
                use_local_extrema=False).add_to(map2)

# Display map
map2

### Checking the usage of the stations against their capacity 
1. When stations capacity exceeds once in a day
2. When stations capacity exceeds Thrice in a day
**3. when stations capacity exceeds 40 times in a year**

In [17]:
# Get bike stations with their capacity
response = requests.get('https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information')
bikeshare_stations = pd.DataFrame(json.loads(response.content)['data']['stations'])[['station_id', 'name', 'lat', 'lon', 'capacity']].astype({'station_id': 'int',})
bikeshare_stations = bikeshare_stations.rename(columns={'station_id': 'Station Id', 
                                                        'name': 'Station Name'})

bikeshare_stations_gdf = gpd.GeoDataFrame(bikeshare_stations, geometry=gpd.points_from_xy(bikeshare_stations.lon,bikeshare_stations.lat))
bikeshare_stations_gdf.crs = {'init': 'epsg:4326'}

bikeshare_stations_gdf = bikeshare_stations_gdf.to_crs(epsg=26917)

In [18]:
trips_data['Start Time DT'] = pd.to_datetime(trips_data['Start Time'])
trips_data['End Time DT'] = pd.to_datetime(trips_data['End Time'])

In [19]:
#Get the Hourly Ride Count from the start Station
trips_data['count'] = 1
period = trips_data['Start Time DT'].dt.to_period("H")
start_station_trip = trips_data.groupby([period, 'Start Station Id']).agg({'count': 'sum'})

In [20]:
start_station_trip.groupby(['Start Station Id', 'Start Time DT']).max()

count
Start Station Id Start Time DT          
7000.0           2017-01-01 00:00      1
                 2017-01-01 19:00      1
                 2017-01-02 09:00      2
                 2017-01-02 10:00      1
                 2017-01-02 12:00      1
...                                  ...
7731.0           2022-10-29 11:00      2
                 2022-10-30 07:00      1
                 2022-10-30 12:00      2
                 2022-10-31 14:00      1
7735.0           2022-10-29 07:00      1

[6439881 rows x 1 columns]

In [21]:
station_cap = bikeshare_stations[['Station Id', 'capacity']].set_index('Station Id')
start_station_cap = start_station_trip.join(station_cap, how='inner', on='Start Station Id')

In [22]:
start_station_cap['Capacity Exceeded'] = start_station_cap['count'] > start_station_cap['capacity']

In [23]:
start_station_cap_exceeded = start_station_cap[start_station_cap['Capacity Exceeded']==True]
start_station_cap_exceeded.reset_index(level=-1).nunique()

Start Station Id     257
count                 88
capacity              28
Capacity Exceeded      1
dtype: int64

In [24]:
print('There are 258 start Stations where capacity Exceeded once in a day')

There are 258 start Stations where capacity Exceeded once in a day


In [25]:
# Get the Hourly Ride Count from the End Station
trips_data['count'] = 1
period2 = trips_data['End Time DT'].dt.to_period("H")
end_station_trip = trips_data.groupby([period, 'End Station Id']).agg({'count': 'sum'})

end_station_cap = end_station_trip.join(station_cap, how='inner', on='End Station Id')
end_station_cap['Capacity Exceeded'] = end_station_cap['count'] > end_station_cap['capacity']

end_station_cap_exceeded = end_station_cap[end_station_cap['Capacity Exceeded']==True]

end_station_cap_exceeded.reset_index(level=-1).nunique()

End Station Id       236
count                 96
capacity              28
Capacity Exceeded      1
dtype: int64

In [26]:
print('There are 236 end Stations where capacity Exceeded once in a day')

There are 236 end Stations where capacity Exceeded once in a day


In [27]:
start_station_cap_exceeded = start_station_cap_exceeded.reset_index(level=-1)

In [28]:
end_station_cap_exceeded = end_station_cap_exceeded.reset_index(level=-1)

In [29]:
# Get all stations whose capacity was exceeded thrice in a 24 hr period
def count_by_3(x):
    if x.shape[0] >= 3:
        return True
    else:
        return False
    
def custom_count_day(x):
    if x.shape[0] >= 1:
        return x.shape[0]
    else:
        return None
    
def custom_count_yr(x):
    if x.shape[0] >= 40:
        return x.shape[0]
    else:
        return None

def get_df_with_cap_exceeded_thrice(df_cap_exceeded, key):
    df = df_cap_exceeded
    df = df.groupby([df.index.asfreq("D"), key])['Capacity Exceeded'].agg(count_by_3)
    return df.to_frame()

def get_df_with_cap_exceeded_count(df_cap_exceeded, key, period):
    df = df_cap_exceeded
    if period == 'D':
        df = df.groupby([df.index.asfreq(period), key])['Capacity Exceeded'].agg(custom_count_day)
    elif period == 'Y':
        df = df.groupby([df.index.asfreq(period), key])['Capacity Exceeded'].agg(custom_count_yr)
    return df.to_frame()


df_count = get_df_with_cap_exceeded_count(start_station_cap_exceeded, 'Start Station Id', 'D')
df_count = get_df_with_cap_exceeded_count(df_count.reset_index(level=-1), 'Start Station Id', 'Y')
s_unique_yr = df_count.loc[df_count['Capacity Exceeded'].notnull()].reset_index()['Start Station Id'].unique()

df_count = get_df_with_cap_exceeded_count(end_station_cap_exceeded, 'End Station Id', 'D')
df_count = get_df_with_cap_exceeded_count(df_count.reset_index(level=-1), 'End Station Id', 'Y')
e_unique_yr = df_count.loc[df_count['Capacity Exceeded'].notnull()].reset_index()['End Station Id'].unique()

stations_of_interest_yr = set(s_unique_yr).intersection(e_unique_yr)

print('There are ' + str(len(stations_of_interest_yr)) + ' where stations capacity exceeded 40 times in a year')

start_station_cap_exceeded_thrice = get_df_with_cap_exceeded_thrice(start_station_cap_exceeded, 'Start Station Id')
end_station_cap_exceeded_thrice = get_df_with_cap_exceeded_thrice(end_station_cap_exceeded, 'End Station Id')

start_station_cap_exceeded_thrice = start_station_cap_exceeded_thrice[start_station_cap_exceeded_thrice['Capacity Exceeded']==True]
end_station_cap_exceeded_thrice = end_station_cap_exceeded_thrice[end_station_cap_exceeded_thrice['Capacity Exceeded']==True]


There are 25 where stations capacity exceeded 40 times in a year


In [30]:
stations_of_interest = [int(i) for i in stations_of_interest_yr]
print(stations_of_interest)

[7043, 7171, 7430, 7175, 7059, 7062, 7707, 7078, 7217, 7354, 7226, 7357, 7242, 7245, 7118, 7502, 7253, 7260, 7261, 7399, 7016, 7275, 7284, 7289, 7038]


In [31]:
start_station_cap_exceeded_thrice = start_station_cap_exceeded_thrice.reset_index(level=-1)
end_station_cap_exceeded_thrice = end_station_cap_exceeded_thrice.reset_index(level=-1)
start_unique = start_station_cap_exceeded_thrice['Start Station Id'].unique()
end_unique = end_station_cap_exceeded_thrice['End Station Id'].unique()
stations_of_interest = set(start_unique).intersection(end_unique)
print("There are " + str(len(stations_of_interest)) + " stations where capacity exceed Thrice")

There are 73 stations where capacity exceed Thrice


In [32]:
def is_cap_exceeded(row, stations_of_interest):
    if (row['Station Id'] in stations_of_interest):
        return True
    else:
        return False

bikeshare_stations_gdf['Capacity Exceeded'] = \
bikeshare_stations_gdf.apply(lambda row: is_cap_exceeded(row, stations_of_interest), axis=1)

In [33]:
# Write your code here.
map3 = folium.Map(location=[43.6426, -79.3871], 
                   tiles='cartodbpositron', 
                   zoom_start=10)

# Add points to the map
for idx, row in bikeshare_stations_gdf.to_crs(epsg=4326).iterrows():
    if (row['Capacity Exceeded']):
        Marker([row.geometry.y, row.geometry.x], 
               icon=folium.Icon(color='red')).add_to(map3)        
    else:
        Circle(location=[row.geometry.y, row.geometry.x],
               radius=20,
               color='green').add_to(map3)      

# Display map
map3


####  Now that we know where the station capacity is exceeded, the solutions are either increase the capacity of the existing station and propose new stations at a location such that they are closer to the existing station and nearby TTC station or school which is shown in the Graph Below


In [35]:
# Proposing Stations closer to the Subway stations
subway_stations = gpd.read_file('subway_stations.shp')
subway_stations = subway_stations.to_crs(epsg=26917)

# Get schools
schools = gpd.read_file('TCDSB - Schools.shp')
schools = schools.to_crs(epsg=26917)

bikeshare_stations_buffer = bikeshare_stations_gdf.geometry.buffer(1000)
bike_station_union = bikeshare_stations_buffer.geometry.unary_union
bike_station_union = gpd.GeoDataFrame(geometry=[bike_station_union], crs='EPSG:26917')

bikeshare_stations_gdf_cap_exceeded = bikeshare_stations_gdf[bikeshare_stations_gdf['Capacity Exceeded']]
bikeshare_stations_buffer_cap_exceeded = bikeshare_stations_gdf_cap_exceeded.geometry.buffer(700)
bike_station_union_cap_exceeded = bikeshare_stations_buffer_cap_exceeded.geometry.unary_union
bike_station_union = gpd.GeoDataFrame(geometry=[bike_station_union_cap_exceeded], crs='EPSG:26917')


subway_stations['bike_access'] = subway_stations.apply(
                                    lambda row: bike_station_union.contains(row.geometry), 
                                    axis=1)

subway_stations['should_expand'] = subway_stations.apply(
                                    lambda row: bike_station_union_cap_exceeded.contains(row.geometry), 
                                    axis=1)


map4 = folium.Map(location=[43.6426, -79.3871], 
                   tiles='cartodbpositron', 
                   zoom_start=10)

# Add points to the map
for idx, row in bikeshare_stations_gdf.to_crs(epsg=4326).iterrows():
    if (row['Capacity Exceeded']):
        Circle(location=[row.geometry.y, row.geometry.x],
               radius=20,
               color='red').add_to(map4)
    else:
        Circle(location=[row.geometry.y, row.geometry.x],
               radius=20,
               color='green').add_to(map4)      

for idx, row in subway_stations.to_crs(epsg=4326).iterrows():
    if row['should_expand']:
        Marker([row.geometry.y, row.geometry.x], 
               icon=folium.Icon(color='green'),
               popup=row['STATION']).add_to(map4)
    else:
        Marker([row.geometry.y, row.geometry.x], 
               icon=folium.Icon(color='black'),
               popup=row['STATION']).add_to(map4)
        
        

# Show the map
map4

In [36]:
map5 = folium.Map(location=[43.6426, -79.3871], 
                   tiles='cartodbpositron', 
                   zoom_start=10)

for idx, row in bikeshare_stations_gdf.to_crs(epsg=4326).iterrows():
    if (row['Capacity Exceeded']):
        Circle(location=[row.geometry.y, row.geometry.x],
               radius=20,
               color='red').add_to(map5)
    else:
        Circle(location=[row.geometry.y, row.geometry.x],
               radius=20,
               color='green').add_to(map5)      
        
schools['should_expand'] = schools.apply(
                                    lambda row: bike_station_union_cap_exceeded.contains(row.geometry), 
                                    axis=1)
     
for idx, row in schools.to_crs(epsg=4326).iterrows():
    if row['should_expand']:
        Marker([row.geometry.y, row.geometry.x], 
               icon=folium.Icon(color='green')).add_to(map5)
    else:
        Marker([row.geometry.y, row.geometry.x], 
               icon=folium.Icon(color='orange')).add_to(map5)

map5